<a href="https://colab.research.google.com/github/steinhaug/bark-infinity/blob/main/notebooks/Bark_Infinity_Long_Form_Audio_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Connect and check GPU and runtime
from psutil import virtual_memory
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
ram_gb = virtual_memory().total / 1e9
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU', end="")
elif gpu_info.find('not found') >= 0:
    print('Not connected to a GPU', end="")
else:
    print('GPU Connected', end="")
print(', your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))


## Setup Notebook, Install dependencies
<small>Run both cells to install system and needed functions.  
_If Colab for some reason crashes re-run cell 0.2 before contining._</small>


In [ ]:
#@title 0.1 - Install system
from IPython.display import clear_output
!git clone https://github.com/JonathanFly/bark.git
%cd bark
!pip install -r old_setup_files/requirements-pip.txt
!pip install encodec rich-argparse
!pip install librosa pydub devtools

#clear_output()
#print('Cell completed.')

In [ ]:
 #@title 0.2 - Setup required functions and helpers
import os
import time
from bark_infinity import config
import numpy as np

logger = config.logger
logger.setLevel("WARNING")

from bark_infinity import generation
from bark_infinity import api

import rich
from rich import print
from rich import pretty
from rich.pretty import pprint
from rich import inspect

import librosa
from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display, Audio
from io import BytesIO

# None of this code, just fiddlign with Colab stuff
# Just to save Colab with outputs and float32 wavs are GIGANTO
# actually this doesn't work, the iPython widget converts it back to float32? or I messed up

def display_audio_int16_but(audio_arr_segments, file_name, sample_rate=generation.SAMPLE_RATE,  width='200px'):
    file_name_label = widgets.Label(value=f"Playing: {file_name}")
    file_name_label.layout.width = width
    audio_data_int16 = audio_arr_segments
    if isinstance(audio_data_int16, list):
        audio_data_int16 = np.concatenate(audio_data_int16)

    #audio_data_int16 = np.int16(audio_data_int16 * np.iinfo(np.int16).max)


    audio_widget = Audio(audio_data_int16, rate=sample_rate)
    display(file_name_label, audio_widget)


def on_button_click(button):
    audio_data, sample_rate = librosa.load(button.wav_path, sr=None)
    file_name = os.path.basename(button.wav_path)
    display_audio_int16_but(audio_data,file_name, sample_rate)

def display_wav_files(directory, matchType=".wav"):
    subdirs, wav_files = [], []

    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)

        if os.path.isfile(item_path) and item_path.endswith(matchType):
            wav_files.append(item_path)
        elif os.path.isdir(item_path):
            subdirs.append(item_path)

    wav_files.sort(key=lambda x: os.path.basename(x))

    for wav_file in wav_files:

        filename = os.path.basename(wav_file)
        print(f" {filename}")
        display( Audio(filename=wav_file, rate=generation.SAMPLE_RATE) )
        #button = widgets.Button(description=f"Play {filename}")
        #button.wav_path = wav_file
        #button.on_click(on_button_click)
        #display(button)

    for subdir in sorted(subdirs):
        print(f"<{subdir}>")
        display_wav_files(subdir, matchType)

def display_mp4_files(directory):
    return display_wav_files(directory, '.mp4')


## 1.0 - Gradio App

In [ ]:
#@markdown Run the WebUI with all features.<br>
#@markdown When loaded click the second link to launch WebUI in another window.
!python bark_webui.py --share

## 2.0 - Manual generation


### 2.1 - Choose Bark Models and set Text and Other Generation Options

Required for 3.0 and 4.0

In [ ]:
# Time to complete cell: ca. 3min
generation.OFFLOAD_CPU = False # On your home system set to True probably, but Colab GPU should have plenty of memory for all three models
generation.preload_models() # Optional, will lazy load if not preloaded. First time run in New Colab has to download models

In [ ]:
text = """
Hey, have you heard about this new text-to-audio model called "Bark"?
It's like rain on your wedding day. It's a free ride when you've already paid. It's the good advice that you just didn't take.
And who would've thought? It figures.

Well, life has a funny way of sneaking up on you. When you think everything's okay and everything's going right.
And life has a funny way of helping you out. When you think everything's gone wrong.
And everything blows up in your face.

It's a traffic jam when you're already late. A "No smoking" sign on your cigarette break.
It's like ten thousand spoons when all you need is a knife. It's meeting the man of my dreams.
And then meeting his beautiful wife.

And isn't it ironic? Don't you think? A little too ironic.
And yeah, I really do think.
"""

# For split set split_character_goal_length and split_character_max_length
kwargs = {}

kwargs = config.load_all_defaults()
kwargs['text_prompt'] = text
kwargs['hoarder_mode'] = True
kwargs["output_dir"] = 'bark_samples'
kwargs["history_prompt"] = None
# kwargs["single_starting_seed"] = None #
# If you set seed you might want manually call generation.set_seed(-1) after to disable deterministic generation settings
# I'm not cleaning up after this paramater at the moment and I'm not sure on other side effects
kwargs["stable_mode_interval"] = 1 # 0 for continous, 2,3,4 for mixed
kwargs["split_character_goal_length"] = 90
kwargs["split_character_max_length"] = 130
# kwargs["output_iterations"] = 1
kwargs["add_silence_between_segments"] = .025 # See: https://github.com/suno-ai/bark/blob/main/notebooks/long_form_generation.ipynb but not great for songs or stable_mode_interval 0
kwargs["semantic_min_eos_p"] = 0.05 # 0.20 is default, lower means more likely to stotp


# not sure on overall effect so far from these, but for example:
kwargs["semantic_top_k"] = 50
kwargs["semantic_top_p"] = 0.95

### 3.0 First Attempt

#### 3.2 Before we run, let's double check out settings

In [ ]:
kwargs["dry_run"] = True # Check how the text is being split, don't actually run the model.
full_generation_segments, audio_arr_segments, final_filename_will_be, _ = api.generate_audio_long(**kwargs)

In [ ]:
# that's the output we expect to see, we didn't generate audio yet
# these text segments look a little small small so let's try this instead
kwargs["split_character_goal_length"] = 110
kwargs["split_character_max_length"] = 175

full_generation_segments, audio_arr_segments, final_filename_will_be, _ = api.generate_audio_long(**kwargs)

#### 3.2 Run Bark

In [ ]:
# These segement sizes look better so now so set dry_run to False to run for real
# Because we set hoarder_mode we can see the wav files for each segment in the Colab File Manager

kwargs["dry_run"] = False
full_generation_segments, audio_arr_segments, final_filename_will_be, _ = api.generate_audio_long(**kwargs)

#### 3.3 Save and list files + playbutton

In [ ]:
print(f"Final audiofile: {final_filename_will_be}")
# (we see many wav because we set hoarder_mode, but one file will be the final product
# set hoarder_mode=False if you just want the file wav and aren't in explore mode

# or play here
Audio(np.concatenate(audio_arr_segments), rate=generation.SAMPLE_RATE)


In [ ]:
# because we set hoarder mode we also saved each segement as its own seperate sample with wav

!find "bark_samples/" -name "*.npz"

display_mp4_files("bark_samples/")

### 4.0 Second Attempt. Can we do better?

#### 4.1 settings

In [ ]:
# we used stable_mode_interval = 1, so the history_prompt does not evolve between segments
# even still the voices that are saved for each segment are one-generation different than the original history prompt
# this means they are a *little* bit different, and we may prefer one of them over the original
# for example maybe segment 2 was a little more clear, or had a particular emotion, we could use that segment's version as the speaker
# in the particular run I'm doing now, that segment ended with a little bit an interesting accent. I'm curious if I can bring that out more.

# (should probably rename the file to something sensible though)

kwargs["history_prompt"] = "/content/bark/bark_samples/Hey_have_you_he-23-0714-0743-27-SPK-random/002_Its_the_good_ad-23-0714-0744-51-SPK-random.mp4.npz"

#### 4.2 generate

In [ ]:
kwargs["text_prompt"] = f"I'm speaker number two. I'm the best speaker. Also I'm a free spirit. Let me evolve my voice with every step. Here's my version."
kwargs["text_prompt"] += text
kwargs["stable_mode_interval"] = 0
kwargs["output_dir"] = "speaker_2_test"
kwargs["add_silence_between_segments"] = 0.0 # No silence, fully merge clips

kwargs["semantic_min_eos_p"] = 0.20 # Back to default, let Bark umm and ahh a bit
full_generation_segments, audio_arr_segments, final_filename_will_be, _ = api.generate_audio_long(**kwargs)


#### 4.3 Save and list files + playbutton

In [ ]:
print(f"Final audiofile: {final_filename_will_be}")
Audio(np.concatenate(audio_arr_segments), rate=generation.SAMPLE_RATE)

In [ ]:
# this clip probably got really weird after a full segments, fully feedbacking into itself. So kwargs["stable_mode_interval"] = 3 might be a good compromise

display_mp4_files("speaker_2_test")